<img align="Left" alt="Dados_ao_Cubo" src="../shared_content/DADOS-AO-CUBO-Versão-04-1.png" />
<img align="right" alt="MLNet" src="../shared_content/EJ3KsfJWsAEtDaZ.png" />

## Davi Ramos -> Cientista de Dados 👋
(davi.info@gmail.com)

[![Linkedin Badge](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/davi-ramos/)](https://www.linkedin.com/in/davi-ramos/)
[![Twitter Badge](https://img.shields.io/badge/-Twitter-1DA1F2?style=flat-square&logo=Twitter&logoColor=white&link=https://twitter.com/Daviinfo/)](https://twitter.com/Daviinfo/)
<a href="https://github.com/DaviRamos"><img src="https://img.shields.io/github/followers/DaviRamos.svg?label=GitHub&style=social" alt="GitHub"></a>

# ML.Net - Auto ML

## Este exemplo irá xxxxxx

In [1]:
// Instalar os Pacotes do Nuget

// ML.NET
#r "nuget:Microsoft.ML"  

// ML.NET AutoML
#r "nuget:Microsoft.ML.AutoML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.2

Installed package Microsoft.ML.AutoML version 0.17.2

In [2]:
using System;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;

using static Microsoft.ML.TrainCatalogBase;
using static Microsoft.ML.DataOperationsCatalog;

In [3]:
// Caminho dos arquivos
string housingPath = "./Datasets/Housing/housing.csv";
string ModelPath = @"./HouseModelv3.zip";
private static string LabelColumnName = "Label";
private static uint ExperimentTime = 10;

In [4]:
public class ModelInput
{
    [LoadColumn(0)]
    public float Longitude { get; set; }
    [LoadColumn(1)]
    public float Latitude { get; set; }
    [LoadColumn(2)]
    public float Housing_median_age { get; set; }
    [LoadColumn(3)]
    public float Total_rooms { get; set; }
    [LoadColumn(4)]
    public float Total_bedrooms { get; set; }
    [LoadColumn(5)]
    public float Population { get; set; }
    [LoadColumn(6)]
    public float Households { get; set; }
    [LoadColumn(7)]
    public float Median_income { get; set; }
    [ColumnName("Label"), LoadColumn(8)]
    public float Median_house_value { get; set; }
    [LoadColumn(9)]
    public string Ocean_proximity { get; set; }
}

public class ModelOutput
{
    [ColumnName("Score")]
    public float Score { get; set; }
}

In [5]:
private static void PrintMetrics(RegressionMetrics metrics)
{
    Console.WriteLine($"MeanAbsoluteError: {metrics.MeanAbsoluteError}");
    Console.WriteLine($"MeanSquaredError: {metrics.MeanSquaredError}");
    Console.WriteLine($"RootMeanSquaredError: {metrics.RootMeanSquaredError}");
    Console.WriteLine($"RSquared: {metrics.RSquared}");
}

In [6]:
// Criar o MLContext para ser compartilhado entre os objetos de fluxo de trabalho de criação do modelo
// Definir uma semente aleatória para resultados repetíveis/determinísticos nos vários treinamentos.
MLContext mlContext = new MLContext(seed: 1);

In [7]:
private static IDataView trainDataView;
private static IDataView testDataView;

//Configuração do carregamento de dados         
IDataView fullData = mlContext.Data.LoadFromTextFile<ModelInput>(path: housingPath, hasHeader: true, 
                                                                         separatorChar: ',', allowQuoting: true, allowSparse: false);

// Divide o conjunto de dados em duas partes: TrainingDataset (80%) e TestDataset (20%)
var trainTestData = mlContext.Data.TrainTestSplit(fullData, testFraction: 0.2);
trainDataView = trainTestData.TrainSet;
testDataView = trainTestData.TestSet;

// STEP 2: Run AutoML experiment
Console.WriteLine($"Running AutoML regression experiment for {ExperimentTime} seconds...");
ExperimentResult<RegressionMetrics> experimentResult = mlContext.Auto()
    .CreateRegressionExperiment(ExperimentTime)
    .Execute(trainDataView,  "Label");

Running AutoML regression experiment for 10 seconds...


In [8]:
// STEP 3: Print metric from best model
RunDetail<RegressionMetrics> bestRun = experimentResult.BestRun;
Console.WriteLine($"Total models produced: {experimentResult.RunDetails.Count()}");
Console.WriteLine($"Best model's trainer: {bestRun.TrainerName}");
Console.WriteLine($"--");
Console.WriteLine($"Metrics of best model from validation data --");
Console.WriteLine($"--");
PrintMetrics(bestRun.ValidationMetrics);

// STEP 5: Evaluate test data
IDataView testDataViewWithBestScore = bestRun.Model.Transform(testDataView);
RegressionMetrics testMetrics = mlContext.Regression.Evaluate(testDataViewWithBestScore, labelColumnName: LabelColumnName);
Console.WriteLine($"--");
Console.WriteLine($"Metrics of best model on test data --");
Console.WriteLine($"--");
PrintMetrics(testMetrics);

// STEP 6: Save the best model for later deployment and inferencing
using (FileStream fs = File.Create(ModelPath))
    mlContext.Model.Save(bestRun.Model, trainDataView.Schema, fs);

// STEP 7: Create prediction engine from the best trained model
var predictionEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(bestRun.Model);

// STEP 8: Initialize a new test taxi trip, and get the predicted fare
var sampleData = new ModelInput()
{
    Longitude = -122.23F,
    Latitude = 37.88F,
    Housing_median_age = 41F,
    Total_rooms = 880F,
    Total_bedrooms = 129F,
    Population = 322F,
    Households = 126F,
    Median_income = 8.3252F,
    Ocean_proximity = @"NEAR BAY",
};

var resultprediction = predictionEngine.Predict (sampleData);

Console.WriteLine($"--");
Console.WriteLine("Usando o modelo para fazer predição única - Comparando o valor atual de Median_house_value com o valor previsto de para os dados de amostra de Median_house_value ... \n \n");
Console.WriteLine($"Longitude: {sampleData.Longitude}");
Console.WriteLine($"Latitude: {sampleData.Latitude}");
Console.WriteLine($"Housing_median_age: {sampleData.Housing_median_age}");
Console.WriteLine($"Total_rooms: {sampleData.Total_rooms}");
Console.WriteLine($"Total_bedrooms: {sampleData.Total_bedrooms}");
Console.WriteLine($"Population: {sampleData.Population}");
Console.WriteLine($"Households: {sampleData.Households}");
Console.WriteLine($"Median_income: {sampleData.Median_income}");
Console.WriteLine($"Ocean_proximity: {sampleData.Ocean_proximity}");
Console.WriteLine ($"\n \nO Preço previsto para este tamanho é: {resultprediction.Score} \n \n");
Console.WriteLine ("=============== Fim do processo ===============");

Total models produced: 26
Best model's trainer: FastTreeRegression
--
Metrics of best model from validation data --
--
MeanAbsoluteError: 31970,344956517583
MeanSquaredError: 2264295999,1416245
RootMeanSquaredError: 47584,61935480439
RSquared: 0,8276824825461914
--
Metrics of best model on test data --
--
MeanAbsoluteError: 32457,33227556957
MeanSquaredError: 2377334729,890651
RootMeanSquaredError: 48757,91966327779
RSquared: 0,8231534128038352
--
Usando o modelo para fazer predição única - Comparando o valor atual de Median_house_value com o valor previsto de para os dados de amostra de Median_house_value ... 
 

Longitude: -122,23
Latitude: 37,88
Housing_median_age: 41
Total_rooms: 880
Total_bedrooms: 129
Population: 322
Households: 126
Median_income: 8,3252
Ocean_proximity: NEAR BAY

 
O Preço previsto para este tamanho é: 441550 
 

=============== Fim do processo ===============
